conda install -c pytorch pytorch 

In [1]:
import pandas as pd
import numpy as np
from torch import nn
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
import torch.nn.functional as F

Using TensorFlow backend.


In [4]:
data = pd.read_csv("SMSSpamCollection", delimiter='\t', header=None, names=["outcome", 'message'])

In [5]:
data.head()


,outcome,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data.outcome = data.outcome.map({'ham':0, 'spam':1})


In [7]:
data.head()

,outcome,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
features = data.message.values
labels = data.outcome.values
num_words = 1000

In [9]:
features.shape

(5572,)

In [10]:
labels.shape


(5572,)

In [11]:
t = Tokenizer(num_words=1000)
t.fit_on_texts(features)

In [12]:
features = t.texts_to_matrix(features, mode='tfidf')

In [13]:
features.shape

(5572, 1000)

In [14]:
# Building model
class Model(nn.Module):
    def __init__(self, input, hidden, output):
        super(Model, self).__init__()
        self.l1 = nn.Linear(input, hidden)
        self.l2 = nn.Linear(hidden , hidden)
        self.l3 = nn.Linear(hidden, 2)
    
    def forward(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out    

In [15]:
input = 1000
hidden=100
output = 2

In [16]:
model = Model(input, hidden, output)


In [17]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, shuffle=True, random_state=34)

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [20]:
def train(epochs):
    x_train = Variable(torch.from_numpy(features_train)).float()
    y_train = Variable(torch.from_numpy(labels_train)).long()
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        print ("epoch #",epoch)
        print ("loss: ", loss.item())
        pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
        print ("acc:(%) ", 100*pred/len(x_train))
        loss.backward()
        optimizer.step()


In [21]:
def test(epochs):
    model.eval()
    x_test = Variable(torch.from_numpy(features_test)).float()
    y_test = Variable(torch.from_numpy(labels_test)).long()
    for epoch in range(epochs):
        with torch.no_grad():
            y_pred = model(x_test)
            loss = criterion(y_pred, y_test)
            print ("epoch #",epoch)
            print ("loss: ", loss.item())
            pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
            print ("acc (%): ", 100*pred/len(x_test))

In [22]:
train(100)

epoch # 0
loss:  0.6867676973342896
acc:(%)  tensor(67)
epoch # 1
loss:  0.42427995800971985
acc:(%)  tensor(86)
epoch # 2
loss:  0.25411853194236755
acc:(%)  tensor(86)
epoch # 3
loss:  0.19334715604782104
acc:(%)  tensor(86)
epoch # 4
loss:  0.1545654535293579
acc:(%)  tensor(89)
epoch # 5
loss:  0.12337833642959595
acc:(%)  tensor(96)
epoch # 6
loss:  0.09726635366678238
acc:(%)  tensor(98)
epoch # 7
loss:  0.07385411858558655
acc:(%)  tensor(98)
epoch # 8
loss:  0.052467864006757736
acc:(%)  tensor(99)
epoch # 9
loss:  0.035277724266052246
acc:(%)  tensor(99)
epoch # 10
loss:  0.02435595914721489
acc:(%)  tensor(99)
epoch # 11
loss:  0.0184079147875309
acc:(%)  tensor(99)
epoch # 12
loss:  0.014859743416309357
acc:(%)  tensor(99)
epoch # 13
loss:  0.012325495481491089
acc:(%)  tensor(99)
epoch # 14
loss:  0.010288997553288937
acc:(%)  tensor(99)
epoch # 15
loss:  0.008017820306122303
acc:(%)  tensor(99)
epoch # 16
loss:  0.005681247916072607
acc:(%)  tensor(99)
epoch # 17
loss:  0.

In [23]:
pred = model(torch.from_numpy(features_test).float())

In [24]:
pred

tensor([[ 19.8446, -19.2942],
        [ 47.7765, -46.7456],
        [-46.3387,  49.4713],
        ...,
        [-12.7518,  13.4020],
        [  1.9719,  -2.0008],
        [ 30.1847, -29.3152]], grad_fn=<AddmmBackward>)

In [25]:
pred = torch.max(pred,1)[1]

In [26]:
len(pred)

1393

In [27]:
len(features_test)

1393

In [28]:
pred = pred.data.numpy()

In [29]:
pred

array([0, 0, 1, ..., 1, 0, 0])

In [30]:
labels_test

array([0, 0, 1, ..., 1, 0, 0])

In [31]:
accuracy_score(labels_test, pred)

0.9820531227566404

In [32]:
p_train = model(torch.from_numpy(features_train).float())

In [33]:
p_train = torch.max(p_train,1)[1]

In [34]:
len(p_train)

4179

In [35]:
p_train = p_train.data.numpy()

In [36]:
p_train

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
accuracy_score(labels_train, p_train)

0.9997607083034219

In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
cm = confusion_matrix(labels_test, pred)

In [40]:
print (cm)

[[1189   15]
 [  10  179]]
